# What Factors Are Influencing SAT Results in NYC High Schools?

Intro
- SAT results?

- Data Description + sources
    - [2012 SAT Results](https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4)
    - [Class Size - School-level detail](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)

- Questions?
    - Is there a relation between class size and SAT results?
    - Is there a relation between high school location/neighborhood and SAT results?
    
Exploring the datasets might give us some new leads to understand the factors influencing the scores.

- conclusion

## Data Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.display.max_columns = 150 # Avoid truncated columns

### Load Datasets

In [2]:
sat_df = pd.read_csv('2012_SAT_Results.csv')
sat_df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


In [3]:
class_size_df = pd.read_csv('class_size.csv')
class_size_df.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


### Data Preparation

In [4]:
# Convert to numeric for SAT results dataset
cols = ['Num of SAT Test Takers', 'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score', 'SAT Writing Avg. Score']

for col in cols:
    sat_df[col] = pd.to_numeric(sat_df[col], errors='coerce')
    
# Rows with NaN values are not useful so we remove them
sat_df = sat_df.dropna()

We need to find the DBN which is a combination of CSD and School code in order to merge the dataframes later on:

In [5]:
# Compute DBN (CSD must be padded)
class_size_df['DBN'] = class_size_df['CSD'].astype(str).str.zfill(2) + class_size_df['SCHOOL CODE']

Only grades 09-12 in general program are concerned with SAT so we need to filter the dataset accordingly:

In [9]:
class_size_df = class_size_df[(class_size_df['GRADE '] == '09-12') & (class_size_df["PROGRAM TYPE"] == "GEN ED")]

Because there are more than one class per school we need to aggregate the dataframe per DBN:

In [10]:
class_size_df = class_size_df.groupby('DBN').mean()
class_size_df.head()

,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
DBN,,,,,,,
01M292,1,88.0000,4.000000,22.564286,18.50,26.571429,NaN
01M332,1,46.0000,2.000000,22.000000,21.00,23.500000,NaN
01M378,1,33.0000,1.000000,33.000000,33.00,33.000000,NaN
01M448,1,105.6875,4.750000,22.231250,18.25,27.062500,NaN
01M450,1,57.6000,2.733333,21.200000,19.40,22.866667,NaN


## Merge Datasets

In [12]:
# Merge with class size
combined_df = sat_df.merge(class_size_df, on='DBN', how='left')